In [1]:
import tensorflow as tf
import numpy as np

2025-12-04 16:10:33.841004: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

W0000 00:00:1764832237.686285   22892 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

Most importantly, all sorts of tensor operations are available:

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

Note that writing t + 10 is equivalent to calling tf.add(t, 10) (indeed, Python
calls the magic method t.__add__(10), which just calls tf.add(t, 10)). Other
operators, like - and *, are also supported. The @ operator was added in
Python 3.5, for matrix multiplication: it is equivalent to calling the
tf.matmul() function

A tensor can also hold a scalar value. In this case, the shape is empty:

In [11]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

# Tensors and Numpy
Tensors play nice with NumPy: you can create a tensor from a NumPy array,
and vice versa. You can even apply TensorFlow operations to NumPy arrays
and NumPy operations to tensors:


In [12]:
a = np.array([2., 4., 5.])

In [13]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [14]:
t.numpy() # or np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

Notice that NumPy uses 64-bit precision by default, while TensorFlow uses 32-bit. This is
because 32-bit precision is generally more than enough for neural networks, plus it runs
faster and uses less RAM. So when you create a tensor from a NumPy array, make sure to
set dtype=tf.float32.

# Type Conversions
Type conversions can significantly hurt performance, and they can easily go
unnoticed when they are done automatically. To avoid this, TensorFlow does
not perform any type conversions automatically: it just raises an exception if
you try to execute an operation on tensors with incompatible types. For
example, you cannot add a float tensor and an integer tensor, and you cannot
even add a 32-bit float and a 64-bit float:

In [17]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: 

In [ ]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: 

This may be a bit annoying at first, but remember that it’s for a good cause!
And of course you can use tf.cast() when you really need to convert types:

In [18]:
t2 = tf.constant(40., dtype=tf.float64)

In [19]:
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

# Varialbles
The tf.Tensor values we’ve seen so far are immutable: we cannot modify
them. This means that we cannot use regular tensors to implement weights in
a neural network, since they need to be tweaked by backpropagation. Plus,
other parameters may also need to change over time (e.g., a momentum
optimizer keeps track of past gradients). What we need is a tf.Variable:

In [20]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [21]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

# Customizing Models and Training Algorithms
You’ll start by creating a custom loss function, which is a straightforward and
common use case.

## Custom Loss Function
The Huber loss is available in Keras (just use
an instance of the tf.keras.losses.Huber class), but let’s pretend it’s not there.
To implement it, just create a function that takes the labels and the model’s
predictions as arguments, and uses TensorFlow operations to compute a
tensor containing all the losses (one per sample):

In [22]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [23]:
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, [...])

NameError: name 'model' is not defined

## Custom Activation Functions, Initializers, Regularizers, and Constraints
Most Keras functionalities, such as losses, regularizers, constraints,
initializers, metrics, activation functions, layers, and even full models, can be
customized in much the same way. Most of the time, you will just need to
write a simple function with the appropriate inputs and outputs. Here are
examples of a custom activation function (equivalent to
tf.keras.activations.softplus() or tf.nn.softplus()), a custom Glorot initializer
(equivalent to tf.keras.initializers.glorot_normal()), a custom ℓ1 regularizer
(equivalent to tf.keras.regularizers.l1(0.01)), and a custom constraint that
ensures weights are all positive (equivalent to tf.keras.con⁠ straints.nonneg()
or tf.nn.relu()):

In [24]:
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

In [25]:
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

In [26]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [27]:
def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

As you can see, the arguments depend on the type of custom function. These
custom functions can then be used normally, as shown here:

In [28]:
layer = tf.keras.layers.Dense(1, 
                    activation=my_softplus,
                    kernel_initializer=my_glorot_initializer,
                    kernel_regularizer=my_l1_regularizer,
                    kernel_constraint=my_positive_weights
                    )


 # Loading and Preprocessing Data with TensorFlow
 when training TensorFlow models on large datasets, you may
prefer to use TensorFlow’s own data loading and preprocessing API, called
tf.data. It is capable of loading and preprocessing data extremely efficiently,
reading from multiple files in parallel using multithreading and queuing,
shuffling and batching samples, and more. Plus, it can do all of this on the fly
—it loads and preprocesses the next batch of data across multiple CPU cores,
while your GPUs or TPUs are busy training the current batch of data.

The tf.data API lets you handle datasets that don’t fit in memory, and it
allows you to make full use of your hardware resources, thereby speeding up
training. Off the shelf, the tf.data API can read from text files (such as CSV
files), binary files with fixed-size records, and binary files that use
TensorFlow’s TFRecord format, which supports records of varying sizes.

TFRecord is a flexible and efficient binary format usually containing protocol
buffers (an open source binary format). The tf.data API also has support for
reading from SQL databases. Moreover, many open source extensions are
available to read from all sorts of data sources, such as Google’s BigQuery
service (see https://tensorflow.org/io).

## The tf.data API
The whole tf.data API revolves around the concept of a tf.data.Dataset: this
represents a sequence of data items. Usually you will use datasets that
gradually read data from disk, but for simplicity let’s create a dataset from a
simple data tensor using tf.data.Dataset.from_tensor_slices():

In [30]:
X = tf.range(10) # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(X)

In [31]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [32]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


2025-12-04 16:13:43.308087: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


The tf.data API is a streaming API: you can very efficiently iterate through a dataset’s
items, but the API is not designed for indexing or slicing.

In [33]:
X_nested = {"a": ([1, 2, 3], [4, 5, 6]), "b": [7, 8, 9]}
dataset = tf.data.Dataset.from_tensor_slices(X_nested)

In [ ]:
for item in dataset:
    print(item)

{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


2025-12-04 16:14:23.710389: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Chaining Transformations

In [ ]:
# 687